# Pre-processing pipeline for spikeglx sessions, zebra finch
- For every run in the session:
 - Load the recordings
 - Extract wav chan with micrhopohone and make a wav chan with the nidq syn signal
 - Get the sync events for the nidq sync channel
 
 - Do bout detection
 
In another notebook, bout detection is curated
- Left to decide where to:
    - Sort spikes
    - Sync the spikes/lfp/nidq
    - make and plot 'bout rasters'

In [1]:
%matplotlib inline
import os
import glob
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal
import traceback
import warnings

from matplotlib import pyplot as plt
from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util import sglxutil as sglu
from ceciestunepipe.util import rigutil as ru
from ceciestunepipe.util import wavutil as wu
from ceciestunepipe.util import syncutil as su

from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb

from ceciestunepipe.util.spikeextractors import preprocess as pre
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import readSGLX as rsgl
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import spikeglxrecordingextractor as sglex

In [3]:
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
logger.info('all modules loaded')

2022-07-16 10:44:54,667 root         INFO     all modules loaded


## Session parameters and raw files

#### list all the sessions for this bird

In [4]:
bird = 's_b1555_22'
all_bird_sess = et.list_sessions(bird)
logger.info('all sessions for bird are {}'.format(all_bird_sess))

2022-07-16 10:44:57,991 root         INFO     all sessions for bird are ['2022-03-25', '2022-03-26', '2022-03-27', '2022-03-28', '2022-03-29', '2022-03-30', '2022-03-31', '2022-04-01', '2022-04-02', '2022-04-03', '2022-04-04', '2022-04-05', '2022-04-06', '2022-04-07', '2022-04-08', '2022-04-09', '2022-04-10', '2022-04-11', '2022-04-12', '2022-04-13', '2022-04-14', '2022-04-15', '2022-04-16', '2022-04-17', '2022-04-18', '2022-04-19', '2022-04-20', '2022-04-21', '2022-04-22', '2022-04-23', '2022-04-24', '2022-04-25', '2022-04-26', '2022-04-27', '2022-04-28', '2022-04-29', '2022-04-30', '2022-05-01', '2022-05-02', '2022-05-03', '2022-05-04', '2022-05-05', '2022-05-06', '2022-05-07', '2022-05-08', '2022-05-09', '2022-05-10', '2022-05-11', '2022-05-12', '2022-05-13', '2022-05-14', '2022-05-15', '2022-05-16', '2022-05-17', '2022-05-18', '2022-05-19', '2022-05-20', '2022-05-21', '2022-05-22', '2022-05-23', '2022-05-24', '2022-05-25', '2022-05-26', '2022-05-27', '2022-05-28', '2022-05-29', '20

### set up bird and sessions parameters
this will define:
- locations of files (for the bird)
- signals and channels to look for in the metadata of the files and in the rig.json parameter file: Note that this have to exist in all of the sessions that will be processed
- 'sess' is unimportant here, but it comes handy if there is need to debug usin a single session

In [5]:
reload(et)
# for one example session

sess_par = {'bird': bird,
           'sess': '2022-07-16',
           'probes': ['probe_0'], #probes of interest
           'mic_list': ['microphone_0', 'microphone_1'], #list of mics of interest, by signal name in rig.json
           'adc_list': ['pressure'],
           'stim_list': ['wav_syn'], # list of adc chans with the stimulus
           'nidq_ttl_list': ['wav_ttl'], # list of TTL signals form the nidq digital inputs to extract (besides the 'sync')
           'sort': 0, #label for this sort instance
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])

ksort_folder = exp_struct['folders']['ksort']
raw_folder = exp_struct['folders']['sglx']

list all the epochs in a session, to check that it is finding what it has to find

In [17]:
sess_epochs = et.list_sgl_epochs(sess_par)
sess_epochs

2022-07-17 01:28:46,429 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1555_22', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-07-16', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-07-16/sglx', 'kwik': '/scratch/earneodo/s_b1555_22/sglx/kwik/2022-07-16', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1555_22/2022-07-16/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1555_22/sglx/msort/2022-07-16', 'ksort': '/scratch/earneodo/s_b1555_22/sglx/ksort/2022-07-16/0', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/sglx/0'}, 'files': {'par': '/scratch/earneodo/s_b1555_22/sglx/ksort/2022-07-16/0/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-07-16/sglx/settings.isf', 'rig': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-07-16/sglx/rig.json', 'kwd': '/scratch/earneodo/s_b155

['0703_g0', '1426_g0']

#### define pre-processing steps for each epoch and for the session

In [18]:
reload(pre)
one_epoch_dict = pre.preprocess_run(sess_par, exp_struct, sess_epochs[1])

2022-07-17 01:28:52,956 ceciestunepipe.util.spikeextractors.preprocess INFO     PREPROCESSING sess 2022-07-16 | epoch 1426_g0
2022-07-17 01:28:52,957 ceciestunepipe.util.spikeextractors.preprocess INFO     getting extractors
2022-07-17 01:28:53,031 ceciestunepipe.util.spikeextractors.preprocess INFO     Got sglx recordings for keys ['nidq', 'lf_0', 'ap_0']
2022-07-17 01:28:53,032 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting microphone channel(s) ['microphone_0', 'microphone_1']
2022-07-17 01:29:05,356 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-07-17 01:29:05,358 ceciestunepipe.util.wavutil INFO     saving (2, 888574516)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/sglx/1426_g0/wav_mic.wav
2022-07-17 01:29:16,026 ceciestunepipe.util.wavutil INFO     saving (2, 888574516)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/sglx/1426_g0/wav_mic.npy
2022-07-17 01:29:20,821 ceciestunepipe.

2022-07-17 01:47:07,650 ceciestunepipe.util.spikeextractors.preprocess INFO     saving events array to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/sglx/1426_g0/ap_0_sync_evt.npy
2022-07-17 01:47:07,733 ceciestunepipe.util.spikeextractors.preprocess INFO     saving t_0 array to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/sglx/1426_g0/ap_0_t0.npy
2022-07-17 01:47:14,705 ceciestunepipe.util.spikeextractors.preprocess INFO     saving sync dict to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/sglx/1426_g0/ap_0_sync_dict.pkl
2022-07-17 01:47:14,725 ceciestunepipe.util.spikeextractors.preprocess INFO     loading syn_dict from /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/sglx/1426_g0/ap_0_sync_dict.pkl
2022-07-17 01:47:14,730 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for nidq
2022-07-17 01:47:14,731 ceciestunepipe.util.spikeextractors.preprocess INFO     File /mnt/sphere/speech_bci/derived_data/s_b1555_22/2

In [11]:
### sequentially process all runs of the sessions
def preprocess_session(sess_par: dict):
    logger.info('pre-process all runs of sess ' + sess_par['sess'])
    # get exp struct
    sess_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])
    # list the epochs
    sess_epochs = et.list_sgl_epochs(sess_par)
    logger.info('found epochs: {}'.format(sess_epochs))
    # preprocess all epochs
    epoch_dict_list = []
    for i_ep, epoch in enumerate(sess_epochs):
        try:
            exp_struct = et.sgl_struct(sess_par, epoch)
            one_epoch_dict = pre.preprocess_run(sess_par, exp_struct, epoch)
            epoch_dict_list.append(one_epoch_dict)
        except Exception as exc:
            warnings.warn('Error in epoch {}'.format(epoch), UserWarning)
            logger.info(traceback.format_exc)
            logger.info(exc)
            logger.info('Session {} epoch {} could not be preprocessed'.format(sess_par['sess'], epoch))
        
    return epoch_dict_list

all_epoch_list = preprocess_session(sess_par)

2022-04-25 11:28:43,627 root         INFO     pre-process all runs of sess 2022-04-24
2022-04-25 11:28:43,629 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1555_22', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-24', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-24/sglx', 'kwik': '/scratch/earneodo/s_b1555_22/sglx/kwik/2022-04-24', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1555_22/2022-04-24/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-24/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1555_22/sglx/msort/2022-04-24', 'ksort': '/scratch/earneodo/s_b1555_22/sglx/ksort/2022-04-24/2', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-24/sglx/2'}, 'files': {'par': '/scratch/earneodo/s_b1555_22/sglx/ksort/2022-04-24/2/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-24/sglx/settings.isf', 'rig': '/mnt/sphere/spee

2022-04-25 11:42:06,385 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-04-25 11:42:06,385 ceciestunepipe.util.wavutil INFO     saving (1, 923100979)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-24/sglx/1846_g0/wav_mic.wav
2022-04-25 11:42:25,829 ceciestunepipe.util.wavutil INFO     saving (1, 923100979)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-24/sglx/1846_g0/wav_mic.npy
2022-04-25 11:42:43,646 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting adc channel(s) ['pressure']
2022-04-25 11:42:43,649 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-04-25 11:42:43,650 ceciestunepipe.util.wavutil INFO     saving (1, 923100979)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-24/sglx/1846_g0/wav_adc.wav
2022-04-25 11:42:47,163 ceciestunepipe.util.wavutil INFO     saving (1, 923100979)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1555_22/20

## Process multiple sessions

In [11]:
sess_list = all_bird_sess
# fist implant, right hemisphere
#sess_list = ['2021-06-24', '2021-06-25', '2021-06-26', '2021-06-27', '2021-06-28', '2021-06-29', '2021-06-30']
sess_list = ['2022-04-16', '2022-04-17', '2022-04-18', '2022-04-24', 
            '2022-04-28', '2022-04-29'] #pressure with HSCDLNN001PDAA3 sensor

In [12]:
all_sess_dict = {}

for one_sess in sess_list[-1:]:
    sess_par['sess'] = one_sess
    preprocess_session(sess_par)

2022-04-19 10:06:06,964 root         INFO     pre-process all runs of sess 2022-04-18
2022-04-19 10:06:06,965 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/s_b1555_22', 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-18', 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-18/sglx', 'kwik': '/scratch/earneodo/s_b1555_22/sglx/kwik/2022-04-18', 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1555_22/2022-04-18/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx', 'tmp': '/scratch/earneodo/tmp', 'msort': '/scratch/earneodo/s_b1555_22/sglx/msort/2022-04-18', 'ksort': '/scratch/earneodo/s_b1555_22/sglx/ksort/2022-04-18/2', 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/2'}, 'files': {'par': '/scratch/earneodo/s_b1555_22/sglx/ksort/2022-04-18/2/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-04-18/sglx/settings.isf', 'rig': '/mnt/sphere/spee

2022-04-19 10:07:40,125 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-04-19 10:07:40,125 ceciestunepipe.util.wavutil INFO     saving (1, 244632116)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/1207_g0/wav_mic.wav
2022-04-19 10:07:44,194 ceciestunepipe.util.wavutil INFO     saving (1, 244632116)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/1207_g0/wav_mic.npy
2022-04-19 10:07:48,937 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting adc channel(s) ['pressure']
2022-04-19 10:07:48,938 ceciestunepipe.util.wavutil INFO     sampling rate 40000
2022-04-19 10:07:48,939 ceciestunepipe.util.wavutil INFO     saving (1, 244632116)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/1207_g0/wav_adc.wav
2022-04-19 10:07:49,881 ceciestunepipe.util.wavutil INFO     saving (1, 244632116)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1555_22/20

2022-04-19 10:09:47,781 ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor.spikeglxrecordingextractor INFO     getting ttl events, chan 2
2022-04-19 10:09:52,723 ceciestunepipe.util.spikeextractors.preprocess INFO     saving (0, 2)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/1530_g0/wav_ttl_evt.npy
2022-04-19 10:09:52,738 ceciestunepipe.util.spikeextractors.preprocess INFO     saving sync nidaq dict to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/1530_g0/wav_sync_dict.pkl
2022-04-19 10:09:52,740 ceciestunepipe.util.spikeextractors.preprocess INFO     Getting sync signals for imec streams: []
2022-04-19 10:09:52,741 ceciestunepipe.util.spikeextractors.preprocess INFO     getting syn patterns for nidq
2022-04-19 10:09:52,742 ceciestunepipe.util.spikeextractors.preprocess INFO     File /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-04-18/sglx/1530_g0/nidq_sync_dict.pkl not found or forced computation, 

## search bouts for those sessions

In [19]:
hparams = { # default parameters work well for starling
    # spectrogram
    'num_freq':1024, #1024# how many channels to use in a spectrogram #
    'preemphasis':0.97, 
    'frame_shift_ms':5, # step size for fft
    'frame_length_ms':10, #128 # frame length for fft FRAME SAMPLES < NUM_FREQ!!!
    'min_level_db':-55, # minimum threshold db for computing spe 
    'ref_level_db':110, # reference db for computing spec
    'sample_rate':None, # sample rate of your data
    
    # spectrograms
    'mel_filter': False, # should a mel filter be used?
    'num_mels':1024, # how many channels to use in the mel-spectrogram
    'fmin': 500, # low frequency cutoff for mel filter
    'fmax': 12000, # high frequency cutoff for mel filter
    
    # spectrogram inversion
    'max_iters':200,
    'griffin_lim_iters':20,
    'power':1.5,

    # Added for the searching
    'read_wav_fun': bs.read_wav_chan, # function for loading the wav_like_stream (has to returns fs, ndarray)
    'file_order_fun': bs.sess_file_id, # function for extracting the file id within the session
    'min_segment': 30, # Minimum length of supra_threshold to consider a 'syllable' (ms)
    'min_silence': 3000, # Minmum distance between groups of syllables to consider separate bouts (ms)
    'min_bout': 3000, # min bout duration (ms)
    'peak_thresh_rms': 0.55, # threshold (rms) for peak acceptance,
    'thresh_rms': 0.25, # threshold for detection of syllables
    'mean_syl_rms_thresh': 0.3, #threshold for acceptance of mean rms across the syllable (relative to rms of the file)
    'max_bout': 180000, #exclude bouts too long
    'l_p_r_thresh': 100, # threshold for n of len_ms/peaks (typycally about 2-3 syllable spans
    
    'waveform_edges': 1000, #get number of ms before and after the edges of the bout for the waveform sample
    
    'bout_auto_file': 'bout_auto.pickle', # extension for saving the auto found files
    'bout_curated_file': 'bout_checked.pickle', #extension for manually curated files (coming soon)
    }

In [20]:
sess_list = ['2022-04-29', #first implant, RH, pressure + RA
            '2022-07-16', #second implant, LH, pressure + RA, but pressure in the first epoch had a HF noise
            ]

In [21]:
#pick sessions to do the bout searching
bout_sessions = sess_list[-1:]
bout_sessions

['2022-07-16']

In [22]:
#all_sessions = sess_list[-1:]
#all_sessions = ['2021-06-15']

for sess in bout_sessions:
    sess_par['sess'] = sess
    sess_bout_pd = sb.get_all_day_bouts(sess_par, hparams, n_jobs=4, ephys_software='sglx')
    sb.save_auto_bouts(sess_bout_pd, sess_par, hparams, software='sglx')
    #sess_bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts')
    #bouts_to_wavs(sess_bout_pd, sess_par, hparams, sess_bouts_folder)

2022-07-17 01:49:36,113 ceciestunepipe.pipeline.searchbout INFO     Will search for bouts through all session s_b1555_22, 2022-07-16
2022-07-17 01:49:36,207 ceciestunepipe.pipeline.searchbout INFO     getting wav files from/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/sglx
2022-07-17 01:49:36,208 ceciestunepipe.pipeline.searchbout INFO     Found 2 files


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  4.1min
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:  8.1min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:  8.1min finished


2022-07-17 01:57:41,883 ceciestunepipe.pipeline.searchbout INFO     getting spectrograms
2022-07-17 01:58:34,553 ceciestunepipe.pipeline.searchbout INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/bouts_sglx/bout_auto.pickle
2022-07-17 01:58:39,909 ceciestunepipe.pipeline.searchbout INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/bouts_sglx/bout_search_params.pickle
2022-07-17 01:58:39,911 ceciestunepipe.util.sound.boutsearch INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/bouts_sglx/bout_search_params.pickle
2022-07-17 01:58:39,919 ceciestunepipe.pipeline.searchbout INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-07-16/bouts_sglx/bout_auto.pickle
2022-07-17 01:58:45,354 ceciestunepipe.pipeline.searchbout INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1555_22/

In [14]:
sess_bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   start_ms        883 non-null    int64  
 1   end_ms          883 non-null    int64  
 2   start_sample    883 non-null    int64  
 3   end_sample      883 non-null    int64  
 4   p_step          883 non-null    object 
 5   rms_p           883 non-null    float64
 6   peak_p          883 non-null    float64
 7   bout_check      883 non-null    bool   
 8   file            883 non-null    object 
 9   len_ms          883 non-null    int64  
 10  syl_in          883 non-null    object 
 11  n_syl           883 non-null    int64  
 12  peaks_p         883 non-null    object 
 13  n_peaks         883 non-null    int64  
 14  l_p_ratio       883 non-null    float64
 15  waveform        883 non-null    object 
 16  confusing       883 non-null    bool   
 17  valid_waveform  883 non-null    boo

In [15]:
np.unique(sess_bout_pd['start_ms']).size

883

# debug

## debug search_bout

In [21]:
## look for a single file
sess = sess_list[0]

exp_struct = et.get_exp_struct(sess_par['bird'], sess, ephys_software='sglx')
source_folder = exp_struct['folders']['derived']
wav_path_list = et.get_sgl_files_epochs(source_folder, file_filter='*wav_mic.wav')
wav_path_list.sort()
logger.info('Found {} files'.format(len(wav_path_list)))
print(wav_path_list)

2021-09-22 15:13:39,371 root         INFO     Found 4 files


['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/1255_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/1740_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/2118_g0/wav_mic.wav']


In [22]:
one_file = wav_path_list[0]

In [ ]:
reload(bs)
epoch_bout_pd, epoch_wav = bs.get_bouts_in_long_file(wav_path_list[0], hparams)

2021-09-22 15:13:45,924 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav


tu vieja file /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav


2021-09-22 15:13:45,962 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 5 chunks


  0%|          | 0/5 [00:00<?, ?it/s]